# HPC Laboratory Work №4: Salt and Pepper

Подключим гугл диск, чтобы иметь возможность загружать файлы

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Импортируем необходимые библиотеки

In [2]:
import numpy as np
import numba
from numba import cuda
import math
from time import time
from PIL import Image
import pathlib


Опишем функцию генерацию шума типа "Salt and Pepper"(Соль и перец) на cpu

In [3]:
def salt_and_pepper_add(image, prob):
  rnd = np.random.rand(image.shape[0], image.shape[1])
  noisy = image.copy()
  noisy[rnd < prob] = 0
  noisy[rnd > 1 - prob] = 255
  return noisy

Зададим медианный фильтр функции "Salt and pepper" для CPU

In [4]:
def median_filter(a):
  b = a.copy()
  start = time()
  for i in range(2, len(a)-1):
    for j in range(2, len(a[i])-1):
      t=[0, 0, 0, 0, 0, 0, 0, 0, 0]
      # t[0] = a[i-1][j-1]
      # t[1] = a[i-1][j]
      # t[2] = a[i-1][j+1]
      # t[3] = a[i][j-1]
      # t[4] = a[i][j]
      # t[5] = a[i][j+1]
      # t[6] = a[i+1][j-1]
      # t[7] = a[i+1][j]
      # t[8] = a[i+1][j+1]
      t[0], t[1], t[2], t[3], t[4], t[5], t[6], t[7], t[8] = a[i-1][j-1], a[i-1][j], a[i-1][j+1], a[i][j-1], a[i][j], a[i][j+1], a[i+1][j-1], a[i+1][j], a[i+1][j+1]
      for k in range(8):
        for l in range(8-k):
          if t[l] > t[l+1]:
            t[l], t[l+1] = t[l+1], t[l]
      b[i][j]=t[(int)(len(t)/2)]
  return b, time()-start

Зададим медианный фильтр функции "Salt and pepper" для GPU

In [5]:
@cuda.jit
def gpu_median_filter(a, b):
  i, j = cuda.grid(2)
  t=cuda.local.array(shape=9, dtype=numba.int64)
  t[0], t[1], t[2], t[3], t[4], t[5], t[6], t[7], t[8] = a[i-1][j-1], a[i-1][j], a[i-1][j+1], a[i][j-1], a[i][j], a[i][j+1], a[i+1][j-1], a[i+1][j], a[i+1][j+1]
  for k in range(8):
    for l in range(8-k):
      if t[l]>t[l+1]:
        t[l], t[l+1] = t[l+1], t[l]
  b[i][j]=t[(int)(len(t)/2)]

Функция создающие помехи, а так же использующие фильтры на cpu и gpu для их устранения.

In [9]:
save_folder = "/content/drive/MyDrive/high_perform_comp_labs"

def experiment(img_name, need_draw):
  im=(Image.open(img_name)).convert('L')
  file_name = img_name.rsplit(".", 1)[0].rsplit("/", 1)[1] # filename without extension
  if need_draw==True:
    display(im)
  img = np.array(im)

  img = salt_and_pepper_add(img, 0.09)
  if need_draw==True:
    display(Image.fromarray(np.uint8(img)))
    img3 = Image.fromarray(np.uint8(img))
    img3.save(save_folder + f'/SAP_image_{file_name}.jpg')

  img1, ctime = median_filter(img)
  if need_draw==True:
    display(Image.fromarray(np.uint8(img1)))
    img4 = Image.fromarray(np.uint8(img1))
    img4.save(save_folder + f'/CPU_image_{file_name}.jpg')

  img2, gtime = prepare_and_exec_gpu(img)
  if need_draw==True:
    display(Image.fromarray(np.uint8(img2)))
    img5 = Image.fromarray(np.uint8(img2))
    img5.save(save_folder +f'/GPU_image_{file_name}.jpg')
  
  n=len(img)*len(img[0])
  print('Количество элементов =', n)
  print('Время на CPU =', ctime)
  print('Время на GPU =', gtime)
  print('Ускорение =', ctime/gtime)
  print()
  return ctime, gtime, ctime/gtime, n

In [6]:
def prepare_and_exec_gpu(a):
  b = a.copy()
  tread_number_block = 32

  a_global = cuda.to_device(a)
  b_global = cuda.to_device(b)
  
  threadsperblock = (tread_number_block, tread_number_block)
  blockspergrid_x = int(math.ceil(a.shape[0] / threadsperblock[1]))
  blockspergrid_y = int(math.ceil(b.shape[1] / threadsperblock[0]))
  blockspergrid = (blockspergrid_x, blockspergrid_y)

  start = time()
  gpu_median_filter[blockspergrid, threadsperblock](a_global, b_global)
  return b_global.copy_to_host(), time()-start 

Запустим функцию на различных изображениях драконов с разным разрешением для каждого из файлов и сохраним полученные данные о скорости работы и ускорении в тектовый файл.


In [11]:
folder = pathlib.Path("/content/drive/MyDrive/dragons")
data = [[],[],[],[]]
with open(save_folder + "results.txt", mode="w") as file:
  for path in folder.iterdir():
    ans = experiment(str(path), True)
    for i in range(len(ans)):
      data[i].append(ans[i])
    ctime, gtime, a1, n1 = data
    file.write(f"{ctime} {gtime} {a1} {n1}\n")
  file.write(str(data))

Output hidden; open in https://colab.research.google.com to view.